position_decoder.ipynb

Script to create a position decoder using multinomial logistic regression

MGC 7/15/2019

In [ ]:
# packages
import os
import pandas
import math
from scipy import io
import numpy
from numpy import squeeze
from sklearn import linear_model
import matplotlib
import matplotlib.pyplot as plt

# data
neuropix_folder = os.path.join('E:\\','Dropbox','Work','neuropixels')
data_dir = os.path.join(neuropix_folder,'data')
# sessions = pandas.read_excel(io=os.path.join(neuropix_folder,'sessions_range_of_contrasts.xlsx'))

# params
params = pandas.read_excel(io=os.path.join(os.getcwd(),'UniversalParams.xlsx'))
dt = 0.2 # time bin for decoding, in ms
every_nth_time_bin = round(dt/float(params['TimeBin']));
dx = 5 # spatial bin for decoding, in cm
track_start = float(params['TrackStart'])
track_end = float(params['TrackEnd'])
numposbins = math.floor((track_end-track_start)/dx)
posx_edges = numpy.linspace(track_start,track_end,numposbins+1)

In [ ]:
# load data from an example session
dat = io.loadmat(os.path.join(data_dir,'npI5_0417_baseline_1.mat'))
post = dat['post']
posx = dat['posx']
trial = dat['trial']
sp = dat['sp'][0][0]

# resample post, posx, and trial according to desired dt
post = post[0::every_nth_time_bin]
posx = posx[0::every_nth_time_bin]
trial = trial[0::every_nth_time_bin]

# get cell ids of "good" units
good_cells = sp['cids'][sp['cgs']==2]

# time bins for position decoding
numtimebins = len(post)
post_edges = squeeze(numpy.linspace(min(post)-dt/2,max(post)+dt/2,numtimebins+1))
post_centers = post_edges[range(0,len(post_edges)-1)]+dt/2

# posx categories for position decoding (binned)
posx_bin = numpy.digitize(posx,posx_edges)

# count spikes in each time bin for each cell
spikecount = numpy.empty((len(good_cells),len(post),))
spikecount[:] = numpy.nan
for cell_idx in range(len(good_cells)):   
    spike_t = sp['st'][sp['clu']==good_cells[cell_idx]]
    spikecount[cell_idx,:] = numpy.histogram(spike_t,bins=post_edges)[0]

In [ ]:
# train the logistic regression model
train_set = squeeze(trial<=30)
X_train = numpy.transpose(spikecount[:,train_set])
y_train = squeeze(posx_bin[train_set])
model = linear_model.LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', max_iter=10000, C = 0.1).fit(X_train, y_train) 

In [ ]:
# test the model 
test_set = squeeze(trial==33)
X_test = numpy.transpose(spikecount[:,test_set])
y_test = squeeze(posx_bin[test_set])
y_pred = model.predict(X_test)

fig, ax = plt.subplots()
ax.plot(range(len(y_test)),y_test)
ax.plot(range(len(y_pred)),y_pred)

ax.set(xlabel='time bin', ylabel='pos bin', title='test trial')
ax.grid()
plt.show()